In [1]:
using SymPy

In [2]:
function sx(s)
    x, y = symbols("x y")
    diff(diff(s, y), y)
end
function sy(s)
    x, y = symbols("x y")
    diff(diff(s, x), x)
end
function sxy(s)
    x, y = symbols("x y")
    -diff(diff(s, x), y)
end
function ΔΔ(f)
    x, y = symbols("x y")
    diff(diff(diff(diff(f, x), x), x), x) + 2 * diff(diff(diff(diff(f, x), x), y), y) + diff(diff(diff(diff(f, y), y), y), y)
end

ΔΔ (generic function with 1 method)

In [3]:
function airy_ansatz(deg)
    x, y = symbols("x y")
    coeffs = Sym[]
    ϕ = 0

    k = 0
    for i in 0:deg
        for j in 0:(deg-i)
            C = symbols("C$k")
            push!(coeffs, C)
            ϕ += C * x^i * y^j
            k += 1
        end
    end

    return ϕ, coeffs
end

airy_ansatz (generic function with 1 method)

In [4]:
function coeff_equations(expr)
    x, y = symbols("x y")
    p = sympy.Poly(expr, x, y)
    return p.coeffs()
end

coeff_equations (generic function with 1 method)

In [5]:
function coeff_equations(expr)
    x, y = symbols("x y")
    return collect(values(expr.as_poly(x, y).as_dict()))
end

coeff_equations (generic function with 1 method)

In [6]:
ϕ, coeffs = airy_ansatz(4)

(C0 + C1*y + C10*x^2*y + C11*x^2*y^2 + C12*x^3 + C13*x^3*y + C14*x^4 + C2*y^2 + C3*y^3 + C4*y^4 + C5*x + C6*x*y + C7*x*y^2 + C8*x*y^3 + C9*x^2, Sym[C0, C1, C2, C3, C4, C5, C6, C7, C8, C9, C10, C11, C12, C13, C14])

In [7]:
ΔΔϕ = ΔΔ(ϕ)

8⋅C₁₁ + 24⋅C₁₄ + 24⋅C₄

In [8]:
eqs = coeff_equations(ΔΔϕ)

1-element Vector{Sym{PyCall.PyObject}}:
 8⋅C₁₁ + 24⋅C₁₄ + 24⋅C₄

In [9]:
x, y = symbols("x y")
b, h = symbols("b h")
N, M, T = symbols("N M T")

(N, M, T)

In [10]:
σx = sx(ϕ)

       2                          2                    
2⋅C₁₁⋅x  + 2⋅C₂ + 6⋅C₃⋅y + 12⋅C₄⋅y  + 2⋅C₇⋅x + 6⋅C₈⋅x⋅y

In [11]:
σy = sy(ϕ)

                 2                                 2       
2⋅C₁₀⋅y + 2⋅C₁₁⋅y  + 6⋅C₁₂⋅x + 6⋅C₁₃⋅x⋅y + 12⋅C₁₄⋅x  + 2⋅C₉

In [12]:
τxy = sxy(ϕ)

                              2                       2
-2⋅C₁₀⋅x - 4⋅C₁₁⋅x⋅y - 3⋅C₁₃⋅x  - C₆ - 2⋅C₇⋅y - 3⋅C₈⋅y 

In [13]:
expN = integrate(σx, (y, -h / 2, h / 2))
expN = expN(x => b)
eqs = vcat(eqs, [expN - N])

2-element Vector{Sym{PyCall.PyObject}}:
                     8⋅C₁₁ + 24⋅C₁₄ + 24⋅C₄
 C4*h^3 - N + h*(2*C11*b^2 + 2*C2 + 2*C7*b)

In [14]:
expM = integrate(y * σx, (y, -h / 2, h / 2))
expM = expM(x => b)
eqs = vcat(eqs, [expM])

3-element Vector{Sym{PyCall.PyObject}}:
                     8⋅C₁₁ + 24⋅C₁₄ + 24⋅C₄
 C4*h^3 - N + h*(2*C11*b^2 + 2*C2 + 2*C7*b)
                      h^3*(2*C3 + 2*C8*b)/4

In [15]:
expT = integrate(τxy, (y, -h / 2, h / 2))
expT = expT(x => b)
eqs = vcat(eqs, [expT])

4-element Vector{Sym{PyCall.PyObject}}:
                     8⋅C₁₁ + 24⋅C₁₄ + 24⋅C₄
 C4*h^3 - N + h*(2*C11*b^2 + 2*C2 + 2*C7*b)
                      h^3*(2*C3 + 2*C8*b)/4
  -C8*h^3/4 + h*(-2*C10*b - 3*C13*b^2 - C6)

In [16]:
s1 = σy(y => h / 2)
eqs = vcat(eqs, coeff_equations(s1))

7-element Vector{Sym{PyCall.PyObject}}:
                     8⋅C₁₁ + 24⋅C₁₄ + 24⋅C₄
 C4*h^3 - N + h*(2*C11*b^2 + 2*C2 + 2*C7*b)
                      h^3*(2*C3 + 2*C8*b)/4
  -C8*h^3/4 + h*(-2*C10*b - 3*C13*b^2 - C6)
                   C10*h + C11*h^2/2 + 2*C9
                            6⋅C₁₂ + 3⋅C₁₃⋅h
                                     12⋅C₁₄

In [17]:
s1 = σy(y => -h / 2)
eqs = vcat(eqs, coeff_equations(s1))

10-element Vector{Sym{PyCall.PyObject}}:
                     8⋅C₁₁ + 24⋅C₁₄ + 24⋅C₄
 C4*h^3 - N + h*(2*C11*b^2 + 2*C2 + 2*C7*b)
                      h^3*(2*C3 + 2*C8*b)/4
  -C8*h^3/4 + h*(-2*C10*b - 3*C13*b^2 - C6)
                   C10*h + C11*h^2/2 + 2*C9
                            6⋅C₁₂ + 3⋅C₁₃⋅h
                                     12⋅C₁₄
                  -C10*h + C11*h^2/2 + 2*C9
                            6⋅C₁₂ - 3⋅C₁₃⋅h
                                     12⋅C₁₄

In [18]:
s1 = τxy(y => h / 2)
eqs = vcat(eqs, coeff_equations(s1))

13-element Vector{Sym{PyCall.PyObject}}:
                     8⋅C₁₁ + 24⋅C₁₄ + 24⋅C₄
 C4*h^3 - N + h*(2*C11*b^2 + 2*C2 + 2*C7*b)
                      h^3*(2*C3 + 2*C8*b)/4
  -C8*h^3/4 + h*(-2*C10*b - 3*C13*b^2 - C6)
                   C10*h + C11*h^2/2 + 2*C9
                            6⋅C₁₂ + 3⋅C₁₃⋅h
                                     12⋅C₁₄
                  -C10*h + C11*h^2/2 + 2*C9
                            6⋅C₁₂ - 3⋅C₁₃⋅h
                                     12⋅C₁₄
                    -C6 - C7*h - 3*C8*h^2/4
                           -2⋅C₁₀ - 2⋅C₁₁⋅h
                                     -3⋅C₁₃

In [19]:
s1 = τxy(y => -h / 2)
eqs = vcat(eqs, coeff_equations(s1))

16-element Vector{Sym{PyCall.PyObject}}:
                     8⋅C₁₁ + 24⋅C₁₄ + 24⋅C₄
 C4*h^3 - N + h*(2*C11*b^2 + 2*C2 + 2*C7*b)
                      h^3*(2*C3 + 2*C8*b)/4
  -C8*h^3/4 + h*(-2*C10*b - 3*C13*b^2 - C6)
                   C10*h + C11*h^2/2 + 2*C9
                            6⋅C₁₂ + 3⋅C₁₃⋅h
                                     12⋅C₁₄
                  -C10*h + C11*h^2/2 + 2*C9
                            6⋅C₁₂ - 3⋅C₁₃⋅h
                                     12⋅C₁₄
                    -C6 - C7*h - 3*C8*h^2/4
                           -2⋅C₁₀ - 2⋅C₁₁⋅h
                                     -3⋅C₁₃
                    -C6 + C7*h - 3*C8*h^2/4
                           -2⋅C₁₀ + 2⋅C₁₁⋅h
                                     -3⋅C₁₃

In [20]:
sol = solve(eqs, coeffs)

Dict{Sym{PyCall.PyObject}, Sym{PyCall.PyObject}} with 12 entries:
  C6  => 0
  C13 => 0
  C7  => 0
  C9  => 0
  C8  => 0
  C10 => 0
  C2  => N/(2*h)
  C14 => 0
  C4  => 0
  C3  => 0
  C12 => 0
  C11 => 0

In [21]:
ϕ_sol = subs(ϕ, sol)

                      2
                   N⋅y 
C₀ + C₁⋅y + C₅⋅x + ────
                   2⋅h 

In [22]:
σx = sx(ϕ_sol)
factor(σx)

N
─
h

In [23]:
σy = sy(ϕ_sol)

0

In [24]:
τxy = sxy(ϕ_sol)
simplify(τxy)

0